# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will train your model using the files

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model!

<a id='step1'></a>
## Step 1: Training Setup

do NOT change the lines of code that are not preceded with a TODO statement.

While working on this notebook, you are strongly encouraged to keep `transform_train` at the provided value.  However, when training the model in the next notebook in this sequence, you are welcome (and encouraged!) to tinker with the 

Note that the images in the dataset all have variable height and width, and so your tensor must 


In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import transforms
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math

def to_var(x, volatile=False):
    """ converts a Pytorch Tensor to a variable and moves to GPU if CUDA is available """
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x, volatile=volatile)

## TODO: Select appropriate values for the Python variables below.
batch_size = 128
vocab_threshold = 4
embed_size = 256
hidden_size = 512
num_epochs = 5
save_every = 1

# TODO: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(), 
    transforms.ToTensor(), 
    transforms.Normalize((0.485, 0.456, 0.406), 
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

# Move models to GPU if CUDA is available. 
if torch.cuda.is_available():
    encoder.cuda()
    decoder.cuda()

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO: Specify the learnable parameters of the model.
params = list(decoder.parameters()) + list(encoder.linear.parameters()) 

# TODO: Define the optimizer.
optimizer = torch.optim.Adam(params=params, lr=0.001)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...
Done (t=0.51s)
creating index...


  0%|          | 821/414113 [00:00<00:50, 8207.05it/s]

index created!
Obtaining caption lengths...


100%|██████████| 414113/414113 [00:41<00:00, 10007.18it/s]


<a id='step2'></a>
## Step 2: Train your Model!
- add options for validation

In [ ]:
# (Optional) TODO: Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))

# need to clarify what will happen here ... 
# when resume training ... what will get overwritten, etc

In [2]:
import torch.utils.data as data
import numpy as np
import os

for epoch in range(num_epochs):
    
    for i_step in range(0, total_step):
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler

        # Obtain the batch.
        for batch in data_loader:
            images, captions = batch[0], batch[1]
            break 
        
        # Convert batch of images and captions to Pytorch Variable.
        images = to_var(images, volatile=True)
        captions = to_var(captions)
        
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)
        
        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        
        # Backward pass.
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Print training statistics.
        print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f'
            %(epoch+1, num_epochs, i_step, total_step, loss.data[0], np.exp(loss.data[0]))) 
            
    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' %(epoch+1)))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' %(epoch+1)))

Epoch [1/5], Step [0/3236], Loss: 9.2141, Perplexity: 10037.4266
Epoch [1/5], Step [1/3236], Loss: 9.0689, Perplexity: 8681.1117
Epoch [1/5], Step [2/3236], Loss: 8.8996, Perplexity: 7329.0346
Epoch [1/5], Step [3/3236], Loss: 8.5955, Perplexity: 5407.0760
Epoch [1/5], Step [4/3236], Loss: 8.2678, Perplexity: 3896.2536
Epoch [1/5], Step [5/3236], Loss: 7.6695, Perplexity: 2141.9108
Epoch [1/5], Step [6/3236], Loss: 6.9323, Perplexity: 1024.8324
Epoch [1/5], Step [7/3236], Loss: 5.9402, Perplexity: 380.0042
Epoch [1/5], Step [8/3236], Loss: 5.5904, Perplexity: 267.8499
Epoch [1/5], Step [9/3236], Loss: 5.2861, Perplexity: 197.5807
Epoch [1/5], Step [10/3236], Loss: 4.9650, Perplexity: 143.3015
Epoch [1/5], Step [11/3236], Loss: 4.8474, Perplexity: 127.4116
Epoch [1/5], Step [12/3236], Loss: 4.9447, Perplexity: 140.4223
Epoch [1/5], Step [13/3236], Loss: 4.7437, Perplexity: 114.8541
Epoch [1/5], Step [14/3236], Loss: 4.9061, Perplexity: 135.1164
Epoch [1/5], Step [15/3236], Loss: 4.6917,

Epoch [1/5], Step [130/3236], Loss: 4.2877, Perplexity: 72.7962
Epoch [1/5], Step [131/3236], Loss: 3.7741, Perplexity: 43.5602
Epoch [1/5], Step [132/3236], Loss: 4.1360, Perplexity: 62.5544
Epoch [1/5], Step [133/3236], Loss: 3.7762, Perplexity: 43.6491
Epoch [1/5], Step [134/3236], Loss: 3.7966, Perplexity: 44.5503
Epoch [1/5], Step [135/3236], Loss: 3.6764, Perplexity: 39.5042
Epoch [1/5], Step [136/3236], Loss: 3.4023, Perplexity: 30.0344
Epoch [1/5], Step [137/3236], Loss: 3.6494, Perplexity: 38.4501
Epoch [1/5], Step [138/3236], Loss: 3.6852, Perplexity: 39.8531
Epoch [1/5], Step [139/3236], Loss: 3.7985, Perplexity: 44.6355
Epoch [1/5], Step [140/3236], Loss: 3.6451, Perplexity: 38.2859
Epoch [1/5], Step [141/3236], Loss: 3.6183, Perplexity: 37.2752
Epoch [1/5], Step [142/3236], Loss: 3.4733, Perplexity: 32.2427
Epoch [1/5], Step [143/3236], Loss: 3.5757, Perplexity: 35.7195
Epoch [1/5], Step [144/3236], Loss: 3.4761, Perplexity: 32.3336
Epoch [1/5], Step [145/3236], Loss: 3.60

Epoch [1/5], Step [259/3236], Loss: 3.2303, Perplexity: 25.2866
Epoch [1/5], Step [260/3236], Loss: 3.7294, Perplexity: 41.6536
Epoch [1/5], Step [261/3236], Loss: 4.0398, Perplexity: 56.8131
Epoch [1/5], Step [262/3236], Loss: 3.3441, Perplexity: 28.3337
Epoch [1/5], Step [263/3236], Loss: 3.3499, Perplexity: 28.4999
Epoch [1/5], Step [264/3236], Loss: 3.6971, Perplexity: 40.3292
Epoch [1/5], Step [265/3236], Loss: 3.9976, Perplexity: 54.4698
Epoch [1/5], Step [266/3236], Loss: 3.2552, Perplexity: 25.9250
Epoch [1/5], Step [267/3236], Loss: 3.2078, Perplexity: 24.7251
Epoch [1/5], Step [268/3236], Loss: 3.4028, Perplexity: 30.0482
Epoch [1/5], Step [269/3236], Loss: 3.3819, Perplexity: 29.4252
Epoch [1/5], Step [270/3236], Loss: 3.2454, Perplexity: 25.6718
Epoch [1/5], Step [271/3236], Loss: 3.5883, Perplexity: 36.1742
Epoch [1/5], Step [272/3236], Loss: 3.2967, Perplexity: 27.0224
Epoch [1/5], Step [273/3236], Loss: 3.6120, Perplexity: 37.0394
Epoch [1/5], Step [274/3236], Loss: 3.41

Epoch [1/5], Step [388/3236], Loss: 3.0887, Perplexity: 21.9484
Epoch [1/5], Step [389/3236], Loss: 3.9378, Perplexity: 51.3037
Epoch [1/5], Step [390/3236], Loss: 3.0207, Perplexity: 20.5067
Epoch [1/5], Step [391/3236], Loss: 3.6041, Perplexity: 36.7468
Epoch [1/5], Step [392/3236], Loss: 3.6262, Perplexity: 37.5682
Epoch [1/5], Step [393/3236], Loss: 3.0826, Perplexity: 21.8156
Epoch [1/5], Step [394/3236], Loss: 3.0587, Perplexity: 21.3003
Epoch [1/5], Step [395/3236], Loss: 3.0145, Perplexity: 20.3786
Epoch [1/5], Step [396/3236], Loss: 3.1673, Perplexity: 23.7428
Epoch [1/5], Step [397/3236], Loss: 3.0048, Perplexity: 20.1818
Epoch [1/5], Step [398/3236], Loss: 3.3477, Perplexity: 28.4385
Epoch [1/5], Step [399/3236], Loss: 3.7250, Perplexity: 41.4713
Epoch [1/5], Step [400/3236], Loss: 3.0737, Perplexity: 21.6221
Epoch [1/5], Step [401/3236], Loss: 3.1113, Perplexity: 22.4508
Epoch [1/5], Step [402/3236], Loss: 3.1290, Perplexity: 22.8502
Epoch [1/5], Step [403/3236], Loss: 3.07

Epoch [1/5], Step [517/3236], Loss: 3.5293, Perplexity: 34.1016
Epoch [1/5], Step [518/3236], Loss: 3.1424, Perplexity: 23.1605
Epoch [1/5], Step [519/3236], Loss: 2.9854, Perplexity: 19.7938
Epoch [1/5], Step [520/3236], Loss: 2.9627, Perplexity: 19.3500
Epoch [1/5], Step [521/3236], Loss: 3.2080, Perplexity: 24.7289
Epoch [1/5], Step [522/3236], Loss: 3.0343, Perplexity: 20.7855
Epoch [1/5], Step [523/3236], Loss: 2.9956, Perplexity: 19.9978
Epoch [1/5], Step [524/3236], Loss: 2.9797, Perplexity: 19.6822
Epoch [1/5], Step [525/3236], Loss: 2.9634, Perplexity: 19.3644
Epoch [1/5], Step [526/3236], Loss: 2.8965, Perplexity: 18.1113
Epoch [1/5], Step [527/3236], Loss: 2.9298, Perplexity: 18.7231
Epoch [1/5], Step [528/3236], Loss: 3.1983, Perplexity: 24.4914
Epoch [1/5], Step [529/3236], Loss: 2.9415, Perplexity: 18.9447
Epoch [1/5], Step [530/3236], Loss: 2.8496, Perplexity: 17.2809
Epoch [1/5], Step [531/3236], Loss: 3.4467, Perplexity: 31.3968
Epoch [1/5], Step [532/3236], Loss: 3.01

...

Epoch [5/5], Step [2614/3236], Loss: 2.1215, Perplexity: 8.3439
Epoch [5/5], Step [2615/3236], Loss: 1.8556, Perplexity: 6.3956
Epoch [5/5], Step [2616/3236], Loss: 1.6734, Perplexity: 5.3303
Epoch [5/5], Step [2617/3236], Loss: 1.7692, Perplexity: 5.8662
Epoch [5/5], Step [2618/3236], Loss: 1.7710, Perplexity: 5.8766
Epoch [5/5], Step [2619/3236], Loss: 1.9465, Perplexity: 7.0041
Epoch [5/5], Step [2620/3236], Loss: 1.8031, Perplexity: 6.0685
Epoch [5/5], Step [2621/3236], Loss: 1.8275, Perplexity: 6.2183
Epoch [5/5], Step [2622/3236], Loss: 1.7424, Perplexity: 5.7113
Epoch [5/5], Step [2623/3236], Loss: 1.8740, Perplexity: 6.5145
Epoch [5/5], Step [2624/3236], Loss: 1.9541, Perplexity: 7.0573
Epoch [5/5], Step [2625/3236], Loss: 1.7659, Perplexity: 5.8470
Epoch [5/5], Step [2626/3236], Loss: 1.9912, Perplexity: 7.3244
Epoch [5/5], Step [2627/3236], Loss: 1.6665, Perplexity: 5.2938
Epoch [5/5], Step [2628/3236], Loss: 1.7932, Perplexity: 6.0088
Epoch [5/5], Step [2629/3236], Loss: 1.8

Epoch [5/5], Step [2742/3236], Loss: 1.8382, Perplexity: 6.2852
Epoch [5/5], Step [2743/3236], Loss: 1.8170, Perplexity: 6.1533
Epoch [5/5], Step [2744/3236], Loss: 1.7520, Perplexity: 5.7661
Epoch [5/5], Step [2745/3236], Loss: 1.8518, Perplexity: 6.3716
Epoch [5/5], Step [2746/3236], Loss: 2.1535, Perplexity: 8.6147
Epoch [5/5], Step [2747/3236], Loss: 1.7235, Perplexity: 5.6044
Epoch [5/5], Step [2748/3236], Loss: 1.8400, Perplexity: 6.2963
Epoch [5/5], Step [2749/3236], Loss: 1.9645, Perplexity: 7.1311
Epoch [5/5], Step [2750/3236], Loss: 1.9305, Perplexity: 6.8931
Epoch [5/5], Step [2751/3236], Loss: 1.7487, Perplexity: 5.7470
Epoch [5/5], Step [2752/3236], Loss: 1.8177, Perplexity: 6.1577
Epoch [5/5], Step [2753/3236], Loss: 1.7290, Perplexity: 5.6353
Epoch [5/5], Step [2754/3236], Loss: 1.9064, Perplexity: 6.7286
Epoch [5/5], Step [2755/3236], Loss: 1.7159, Perplexity: 5.5617
Epoch [5/5], Step [2756/3236], Loss: 2.6558, Perplexity: 14.2367
Epoch [5/5], Step [2757/3236], Loss: 1.

Epoch [5/5], Step [2870/3236], Loss: 1.8572, Perplexity: 6.4059
Epoch [5/5], Step [2871/3236], Loss: 1.8056, Perplexity: 6.0835
Epoch [5/5], Step [2872/3236], Loss: 1.7607, Perplexity: 5.8166
Epoch [5/5], Step [2873/3236], Loss: 1.7773, Perplexity: 5.9137
Epoch [5/5], Step [2874/3236], Loss: 1.9695, Perplexity: 7.1672
Epoch [5/5], Step [2875/3236], Loss: 1.7700, Perplexity: 5.8707
Epoch [5/5], Step [2876/3236], Loss: 1.7596, Perplexity: 5.8098
Epoch [5/5], Step [2877/3236], Loss: 1.8471, Perplexity: 6.3413
Epoch [5/5], Step [2878/3236], Loss: 1.7822, Perplexity: 5.9427
Epoch [5/5], Step [2879/3236], Loss: 1.8232, Perplexity: 6.1916
Epoch [5/5], Step [2880/3236], Loss: 1.6844, Perplexity: 5.3892
Epoch [5/5], Step [2881/3236], Loss: 1.6973, Perplexity: 5.4591
Epoch [5/5], Step [2882/3236], Loss: 1.9192, Perplexity: 6.8158
Epoch [5/5], Step [2883/3236], Loss: 1.6969, Perplexity: 5.4572
Epoch [5/5], Step [2884/3236], Loss: 1.7682, Perplexity: 5.8602
Epoch [5/5], Step [2885/3236], Loss: 1.8

Epoch [5/5], Step [2998/3236], Loss: 1.8521, Perplexity: 6.3734
Epoch [5/5], Step [2999/3236], Loss: 1.7871, Perplexity: 5.9719
Epoch [5/5], Step [3000/3236], Loss: 1.6904, Perplexity: 5.4215
Epoch [5/5], Step [3001/3236], Loss: 1.7972, Perplexity: 6.0325
Epoch [5/5], Step [3002/3236], Loss: 2.2483, Perplexity: 9.4712
Epoch [5/5], Step [3003/3236], Loss: 1.7820, Perplexity: 5.9416
Epoch [5/5], Step [3004/3236], Loss: 1.7311, Perplexity: 5.6471
Epoch [5/5], Step [3005/3236], Loss: 1.7174, Perplexity: 5.5699
Epoch [5/5], Step [3006/3236], Loss: 1.7881, Perplexity: 5.9782
Epoch [5/5], Step [3007/3236], Loss: 1.7860, Perplexity: 5.9654
Epoch [5/5], Step [3008/3236], Loss: 1.7611, Perplexity: 5.8191
Epoch [5/5], Step [3009/3236], Loss: 1.9377, Perplexity: 6.9428
Epoch [5/5], Step [3010/3236], Loss: 1.7232, Perplexity: 5.6026
Epoch [5/5], Step [3011/3236], Loss: 2.4499, Perplexity: 11.5873
Epoch [5/5], Step [3012/3236], Loss: 1.8487, Perplexity: 6.3516
Epoch [5/5], Step [3013/3236], Loss: 1.

Epoch [5/5], Step [3126/3236], Loss: 1.9259, Perplexity: 6.8613
Epoch [5/5], Step [3127/3236], Loss: 1.7614, Perplexity: 5.8207
Epoch [5/5], Step [3128/3236], Loss: 1.7728, Perplexity: 5.8872
Epoch [5/5], Step [3129/3236], Loss: 1.7809, Perplexity: 5.9354
Epoch [5/5], Step [3130/3236], Loss: 1.7660, Perplexity: 5.8474
Epoch [5/5], Step [3131/3236], Loss: 2.4904, Perplexity: 12.0666
Epoch [5/5], Step [3132/3236], Loss: 2.1004, Perplexity: 8.1692
Epoch [5/5], Step [3133/3236], Loss: 1.8316, Perplexity: 6.2439
Epoch [5/5], Step [3134/3236], Loss: 1.7435, Perplexity: 5.7171
Epoch [5/5], Step [3135/3236], Loss: 1.7988, Perplexity: 6.0422
Epoch [5/5], Step [3136/3236], Loss: 1.9956, Perplexity: 7.3563
Epoch [5/5], Step [3137/3236], Loss: 1.7150, Perplexity: 5.5567
Epoch [5/5], Step [3138/3236], Loss: 1.9861, Perplexity: 7.2872
Epoch [5/5], Step [3139/3236], Loss: 1.6798, Perplexity: 5.3646
Epoch [5/5], Step [3140/3236], Loss: 1.7433, Perplexity: 5.7164
Epoch [5/5], Step [3141/3236], Loss: 1.